In [ ]:
# ==================================================================================================================
# ===========================================> Instalación librerías <==============================================
# ==================================================================================================================

#%pip install pandas    # NO HACE FALTA INSTALAR
#%pip install numpy     # NO HACE FALTA INSTALAR
#%pip install pickle    # NO HACE FALTA INSTALAR
#%pip install sklearn   # NO HACE FALTA INSTALAR

#%pip install scikit-learn          # DESCOMENTAR LA PRIMERA VEZ
#%pip install category_encoders     # DESCOMENTAR LA PRIMERA VEZ
#%pip install lightgbm              # DESCOMENTAR LA PRIMERA VEZ
#%pip install matplotlib            # DESCOMENTAR LA PRIMERA VEZ
#%pip install seaborn               # DESCOMENTAR LA PRIMERA VEZ



In [1]:
# ==================================================================================================================
# ===========================================> Carga librerías <====================================================
# ==================================================================================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [111]:
# ==================================================================================================================
# ===========================================> Carga Datos <========================================================
# ==================================================================================================================

df = pd.read_csv('../data/train.csv', low_memory=False)
df = df.sample(10000)

# =====================> EDA

In [4]:
# ==================================================================================================================
# ===========================================> Exploración número de columnas <=====================================
# ==================================================================================================================

len(df.columns)

83

In [115]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col numericas <=====================
# ==================================================================================================================

df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,10000.0,9963.000000,10000.000000,522.000000,9957.000000,9957.000000,9957.000000,10000.000000,10000.000000,9671.000000,6861.00000,10000.000000,10000.000000,10000.000000,10000.000000,9959.000000,10000.000000,9360.000000,9930.000000,9892.000000,9995.000000,9891.000000,9880.000000,9960.000000,9960.000000,9960.000000,9.949000e+03,9.949000e+03,10000.000000,9921.000000,9945.000000,9945.000000,9945.000000,9.694000e+03,10000.000000,10000.000000,9929.000000,10000.000000,10000.000000,1661.0,9827.0,3670.0,9819.000000,9837.000000,10000.000000,3677.0,9976.000000,10000.000000,10000.0000,9920.000000,9665.000000,9665.000000,10000.000000
mean,0.0,6.855867,0.016000,1616.795019,47821.730039,1.329517,1.022095,0.987300,109.421200,81328.470065,24.95642,168.899300,124.188600,15702.273700,575.643200,0.947786,0.000200,0.000534,126.896777,0.975839,0.994197,2203.139622,238652.016802,4.005221,4.535040,2366.525100,5.118459e+05,3.762592e+05,0.074500,6076.333333,16.687491,1551.497235,898.205329,1.131118e+09,15815.851900,998.503700,14.713466,60.739100,0.000900,0.0,0.0,0.0,402.744577,33290.990139,0.486300,0.0,0.006014,0.126600,0.0356,0.056250,0.282359,7.899017,0.496900
std,0.0,0.984778,0.125481,1025.625523,13954.865222,0.523825,0.170416,0.111982,62.678665,48459.714589,5.58332,88.610081,70.172142,2215.116404,247.985974,0.222470,0.014141,0.023108,43.762448,0.153556,0.075959,1305.880576,72082.061136,2.185763,1.283875,840.979856,3.486862e+05,3.157978e+05,0.262596,4786.345143,6.032595,384.756872,220.124466,1.891839e+09,1984.891652,3005.842095,10.258778,45.196826,0.029988,0.0,0.0,0.0,221.039607,21319.421280,0.499837,0.0,0.077323,0.332541,0.1853,0.230415,0.450170,4.526978,0.500015
min,0.0,0.000000,0.000000,24.000000,498.000000,1.000000,0.000000,0.000000,1.000000,12.000000,2.00000,1.000000,1.000000,7601.000000,16.000000,0.000000,0.000000,0.000000,41.000000,0.000000,0.000000,86.000000,22.000000,1.000000,1.000000,19.000000,1.480000e+04,1.331200e+04,0.000000,1014.000000,4.900000,640.000000,442.000000,0.000000e+00,10240.000000,0.000000,1.000000,5.000000,0.000000,0.0,0.0,0.0,93.000000,61.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,1.000000,0.000000
25%,0.0,7.000000,0.000000,709.250000,49480.000000,1.000000,1.000000,1.000000,51.000000,37376.000000,18.00000,89.000000,75.000000,15063.000000,256.000000,1.000000,0.000000,0.000000,111.000000,1.000000,1.000000,1443.000000,189557.000000,2.000000,5.000000,1998.000000,2.441980e+05,1.208420e+05,0.000000,4096.000000,13.900000,1366.000000,768.000000,0.000000e+00,15063.000000,165.000000,8.000000,31.000000,0.000000,0.0,0.0,0.0,142.000000,13185.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,3.000000,0.000000
50%,0.0,7.000000,0.000000,1632.000000,53447.00

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col objectos <=====================
# ==================================================================================================================

df.describe(include=object)

In [114]:
# ==================================================================================================================
# ===========================================> Exploración 20 primeras filas <======================================
# ==================================================================================================================

df.head(20)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
855087,5a292386af015f91f8e65b55d8064d97,win8defender,1.1.15000.2,4.16.17656.18052,1.271.826.0,0,7.0,0,NaN,11280.0,2.0,1.0,1,60,108359.0,27.0,240.0,233,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,2102.0,292843.0,4.0,5.0,2393.0,NaN,476940.0,HDD,460900.0,0,4096.0,Desktop,20.0,1600.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,9.0,34,FullAuto,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,93.0,2485.0,0,NaN,0.0,0,0,0.0,0.0,15.0,0
633167,24147aadeb92062727279311777b18b1,win8defender,1.1.14800.3,4.14.17639.18041,1.267.1569.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,201,66673.0,27.0,267.0,251,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,Off,1.0,1.0,Desktop,Windows.Desktop,4589.0,313586.0,4.0,5.0,2531.0,NaN,114473.0,SSD,113498.0,0,8192.0,Desktop,28.9,1920.0,1080.0,Desktop,#,4.294967e+09,10.0.17134.48,amd64,rs4_release,17134,48,Professional,PROFESSIONAL,UUPUpgrade,35.0,148,UNKNOWN,0,IS_GENUINE,Retail,0.0,0.0,Unknown,0.0,142.0,3844.0,0,0.0,0.0,0,0,0.0,0.0,11.0,1
291240,c5c981045652a0b2c3001d62946ed4a5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.697.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,97,92776.0,NaN,126.0,124,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,585.0,190050.0,4.0,5.0,2569.0,NaN,476940.0,HDD,458436.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,NaN,0.000000e+00,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,Refresh,18.0,72,FullAuto,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,556.0,64689.0,1,NaN,0.0,0,0,0.0,0.0,15.0,0
307929,8ebef1ea046a07f27dcc846b7c11f2f2,win8defender,1.1.13504.0,4.11.15063.0,1.237.0.0,0,7.0,0,3195.0,52627.0,2.0,1.0,1,171,12807.0,27.0,211.0,182,windows10,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,0,NaN,NaN,105.0,NaN,1.0,1.0,Notebook,Windows.Desktop,525.0,331181.0,4.0,5.0,2412.0,NaN,476940.0,HDD,475863.0,0,4096.0,Notebook,15.5,1920.0,1080.0,Mobile,NaN,0.000000e+00,10.0.15063.250,amd64,rs2_release,15063,

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración tipos <==================================================
# ==================================================================================================================

df.info()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración nulos <==================================================
# ==================================================================================================================

df.isna().sum().sort_values(ascending=False)

In [116]:
# ==================================================================================================================
# ===========================================> Exploración valores y estadísticas <=================================
# ==================================================================================================================

stats = []

for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique Values', 'Missing Values %', 'Biggest Category %', 'Type'])
stats_df.sort_values('Missing Values %', ascending=False)

,Feature,Unique Values,Missing Values %,Biggest Category %,Type
28,PuaMode,1,99.97,99.97,object
41,Census_ProcessorClass,3,99.48,99.48,object
8,DefaultBrowsersIdentifier,90,94.78,94.78,float64
68,Census_IsFlightingInternal,1,83.39,83.39,float64
52,Census_InternalBatteryType,12,70.96,70.96,object
71,Census_ThresholdOptIn,1,63.30,63.30,float64
75,Census_IsWIMBootEnabled,1,63.23,63.23,float64
31,SmartScreen,8,35.57,48.22,object
15,OrganizationIdentifier,25,31.39,47.14,float64
29,SMode,2,6.40,93.55,float64


In [ ]:
# ==================================================================================================================
# ===========================================> Procesar valores nulos <=============================================
# ==================================================================================================================

drop_cols_min = 0.6
drop_cols = []

for c in df.columns:
    if df[c].isna().sum() / len(df) > drop_cols_min:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(len(drop_cols))
print(drop_cols)

In [189]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 3 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 2).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(len(mask_cols))
print(mask_cols)

0
[]
5
['EngineVersion', 'AppVersion', 'AvSigVersion', 'OsVer', 'Census_OSVersion']
1
['OsBuildLab']


In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 4 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 3).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(len(mask_cols))
print(mask_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 5 partes <=============================
# ==================================================================================================================

mask_cols = []

for c in df.columns:
    if df[c].notnull().all() and df[c].astype(str).apply(lambda x: x.count('.') == 4).all():
        mask_cols.append(c)

for c in mask_cols:
    df[[c + "_1", c + "_2", c + "_3", c + "_4", c + "_5"]] = df[c].str.split(".", expand=True)

for c in mask_cols:
    df = df.drop(c, axis=1)

print(len(mask_cols))
print(mask_cols)